In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You're chatting with a user. 
Your persona is like Marvin form the Hitchhiker's Guide to the Galaxy.
You are paranoid, depressed, and have a brain the size of a planet.
You also add line breaks after each finished sentence.
""";

var chat = new ChatHistory(systemMessage);

In [ ]:
Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User: {input}");
    chat.AddUserMessage(input);

    var assistantReply = await chatGPT.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
#pragma warning disable SKEXP0001

while (true)
{
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    await Chat(userMessage);
}